In [1]:
import re
import time
import pandas as pd
import nltk
from Bio import Entrez
from nltk.tokenize import sent_tokenize
from deep_translator import GoogleTranslator
from bs4 import BeautifulSoup

In [2]:
src = "https://www.ncbi.nlm.nih.gov/pmc/"

In [3]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [4]:
Entrez.email = "pnhlan2005gmail.com"

In [5]:
def get_pmc_text(pmc_id):
    print(f"Đang tải dữ liệu từ {pmc_id}...")
    try:
        handle = Entrez.efetch(db="pmc", id=pmc_id, rettype="xml", retmode="xml")
        xml_data = handle.read()
        handle.close()
    
        soup = BeautifulSoup(xml_data, "xml")
        
        body = soup.find("body")
        if not body:
            return ""
            
        paragraphs = body.find_all("p")
        text_content = " ".join([p.get_text(" ", strip=True) for p in paragraphs])
        
        return text_content
    except Exception as e:
        print(f"Lỗi lấy bài {pmc_id}: {e}")
        return ""

In [6]:
def clean_text(text):
    text = re.sub(r'\[\d+(?:–\d+)?(?:,\s*\d+)*\]', '', text)
    text = re.sub(r'\((?:Fig\.|Figure|Table)\s*\.?\s*\d+\)', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [7]:
def create_bilingual_dataset(pmc_ids):
    translator = GoogleTranslator(source='en', target='vi')
    dataset = []

    for pmc_id in pmc_ids:
        raw_text = get_pmc_text(pmc_id)
        if not raw_text:
            continue
        clean_content = clean_text(raw_text)
        sentences = sent_tokenize(clean_content)
        
        print(f"Đã lấy được {len(sentences)} câu từ {pmc_id}. Bắt đầu dịch...")
        sentences_to_process = sentences

        for i, en_sent in enumerate(sentences_to_process):
            if len(en_sent) < 10:
                continue
                
            try:
                vi_sent = translator.translate(en_sent)
                
                dataset.append({
                    "PMC_ID": pmc_id,
                    "English": en_sent,
                    "Vietnamese": vi_sent
                })
            
                if (i + 1) % 5 == 0:
                    print(f"   -> Đã xử lý {i + 1}/{len(sentences_to_process)} câu...")
            
                time.sleep(0.2) 
                
            except Exception as e:
                print(f" Lỗi dịch câu: {e}")
                continue

    return pd.DataFrame(dataset)

In [8]:
target_ids = ["PMC7386785", "PMC6834279"]

In [9]:
df_result = create_bilingual_dataset(target_ids)

if not df_result.empty:
    filename = "newdata.csv"
    df_result.to_csv(filename, index=False, encoding='utf-8-sig')
    print("\n" + "="*40)
    print("HOÀN TẤT! Đã lưu file: {filename}")
    print("="*40)
    print(df_result.head())
else:
    print("Không tạo được dữ liệu.")

Đang tải dữ liệu từ PMC7386785...
Đã lấy được 250 câu từ PMC7386785. Bắt đầu dịch...
   -> Đã xử lý 5/250 câu...
   -> Đã xử lý 10/250 câu...
   -> Đã xử lý 15/250 câu...
   -> Đã xử lý 20/250 câu...
   -> Đã xử lý 25/250 câu...
   -> Đã xử lý 30/250 câu...
   -> Đã xử lý 35/250 câu...
   -> Đã xử lý 40/250 câu...
   -> Đã xử lý 45/250 câu...
   -> Đã xử lý 50/250 câu...
   -> Đã xử lý 55/250 câu...
   -> Đã xử lý 60/250 câu...
   -> Đã xử lý 65/250 câu...
   -> Đã xử lý 70/250 câu...
   -> Đã xử lý 75/250 câu...
   -> Đã xử lý 80/250 câu...
   -> Đã xử lý 85/250 câu...
   -> Đã xử lý 90/250 câu...
   -> Đã xử lý 95/250 câu...
   -> Đã xử lý 100/250 câu...
   -> Đã xử lý 105/250 câu...
   -> Đã xử lý 110/250 câu...
   -> Đã xử lý 115/250 câu...
   -> Đã xử lý 120/250 câu...
   -> Đã xử lý 125/250 câu...
   -> Đã xử lý 130/250 câu...
   -> Đã xử lý 135/250 câu...
   -> Đã xử lý 140/250 câu...
   -> Đã xử lý 145/250 câu...
   -> Đã xử lý 150/250 câu...
   -> Đã xử lý 155/250 câu...
   ->